In [ ]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('cb_multi_labeled_balanced.csv')

# View the first few rows and the structure of the data
print(df.head())
print(df.info())
print(df.columns)

                                                text              label
0  @ZubearSays Any real nigga isn't letting this ...     ethnicity/race
1  @MoradoSkittle @prolifejewess @DAConsult @Kell...  not_cyberbullying
2        the only thing i wish, i wish a nigga would     ethnicity/race
3  You saudias are not friends of Muslim idiots c...           religion
4  @JaydenT2399 @TractorLaw @holmes_gael @erconge...           religion
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99990 entries, 0 to 99989
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    99990 non-null  object
 1   label   99990 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB
None
Index(['text', 'label'], dtype='object')


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data (run these once)
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Remove @names (mentions)
    text = re.sub(r'@\w+', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation and non-alphabetic characters
    text = re.sub(r'[^a-z\s]', '', text)

    # Remove extra whitespace
    text = text.strip()

    # Tokenize by splitting on whitespace
    tokens = text.split()

    # Remove stopwords and lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # Join tokens back into a string
    return " ".join(tokens)

# Apply text cleaning to the 'text' column of your DataFrame
df['clean_text'] = df['text'].apply(clean_text)

# Inspect the cleaned text
print(df[['clean_text']].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\daksh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\daksh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                          clean_text
0                     real nigga isnt letting happen
1  didnt even mention mexico u far safest place e...
2                        thing wish wish nigga would
3       saudias friend muslim idiot cheapless people
4  yet maga want judah christian muslim believe s...


In [ ]:
# Check for missing values in key columns
print(df.isnull().sum())

df = df.dropna(subset=['text'])

# Remove duplicate rows if any
df = df.drop_duplicates()

print("After cleaning, dataset shape:", df.shape)


text          0
label         0
clean_text    0
dtype: int64
After cleaning, dataset shape: (99989, 3)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# 1. Label-encode the multi-class labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 2. Create a binary label column
#    0 -> 'not_cyberbullying'
#    1 -> all other labels
df['binary_label'] = df['label'].apply(lambda x: 0 if x == 'not_cyberbullying' else 1)

# Inspect the results
print(df[['label', 'binary_label']].head())


               label  binary_label
0     ethnicity/race             1
1  not_cyberbullying             0
2     ethnicity/race             1
3           religion             1
4           religion             1


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the cleaned text data
X = vectorizer.fit_transform(df['clean_text'])

# Labels
y = df['clean_text']
y

0                           real nigga isnt letting happen
1        didnt even mention mexico u far safest place e...
2                              thing wish wish nigga would
3             saudias friend muslim idiot cheapless people
4        yet maga want judah christian muslim believe s...
                               ...                        
99985         thank fuck didnt dance remix peru loveisland
99986    sierra canyon v alemany california high school...
99987                       bitch wasnt flirting snitching
99988                                   like yo nigga call
99989                                       happy cake day
Name: clean_text, Length: 99989, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)


Training set size: (79991, 5000)
Test set size: (19998, 5000)


In [ ]:
# Drop the columns and create a new DataFrame
df_new = df.drop(columns=['text', 'label', 'label_encoded'])

# OR drop them in-place (this modifies 'df' directly)
df.drop(columns=['text', 'label', 'label_encoded'], inplace=True)


,clean_text,binary_label
0,real nigga isnt letting happen,1
1,didnt even mention mexico u far safest place e...,0
2,thing wish wish nigga would,1
3,saudias friend muslim idiot cheapless people,1
4,yet maga want judah christian muslim believe s...,1
...,...,...
99985,thank fuck didnt dance remix peru loveisland,0
99986,sierra canyon v alemany california high school...,0
99987,bitch wasnt flirting snitching,1
99988,like yo nigga call,1


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)
X_train

y_train


Training set size: (79991, 5000)
Test set size: (19998, 5000)


58520                                 love hope get chance
38239    look know picked lottery number even ticket sa...
3806     always sneak charlotte friend call like bitch ...
27925    fit ugly hell cant wait yall admit year httpst...
6006                   welcome clay farm httpstcoqilkaoian
                               ...                        
6265                                            yay adding
54887    isi savvy social medium bc teacher adam ghadan...
76821                          killed dirty hand son bitch
860      excellent academic help team essay pay law ess...
15795                        bitch liar httpstcokkwiufvcuo
Name: clean_text, Length: 79991, dtype: object

In [ ]:
counts = df['binary_label'].value_counts().to_dict()
print("Non-cyberbullying (0):", counts.get(0, 0))
print("Cyberbullying (1):", counts.get(1, 0))


Non-cyberbullying (0): 49999
Cyberbullying (1): 49990


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Load and preprocess your data
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['clean_text'])
y = df['binary_label']
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Choose logistic regression
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
# Evaluation
print("Accuracy score:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


Accuracy score: 0.9922492249224922
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     10124
           1       1.00      0.99      0.99      9874

    accuracy                           0.99     19998
   macro avg       0.99      0.99      0.99     19998
weighted avg       0.99      0.99      0.99     19998

[[10114    10]
 [  145  9729]]
